In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import os
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
from sklearn.metrics import r2_score
from scipy.interpolate import interp1d
import tensorflow_datasets as tfds
from collections import Counter
import scipy.ndimage
from tensorflow.keras.callbacks import ModelCheckpoint
import keras
import gc
import random
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import tarfile
import urllib.request
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torch.nn.functional as F
from scipy.stats import gaussian_kde
from numba import jit
import torch.nn as nn

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

limit = 1000

# bzq modifying
len_x_target = 20
len_y_target = 20
stride_x_target = 10
stride_y_target = 10

# mean, std proportion
alpha = 0.5

bins_size = 30  # 統計採樣數
poly_degree = bins_size - 1
window_size = 1

#target image preprocessing
angle = 0
pixels = 0

In [ ]:
# 設置設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MCDropoutResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.resnet.fc = nn.Sequential(
            nn.Dropout(p=0),
            nn.Linear(self.resnet.fc.in_features, 1000)
        )
        # 複製原始 ResNet50 的 fc 層權重
        original_resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.resnet.fc[1].weight.data = original_resnet.fc.weight.data.clone()
        self.resnet.fc[1].bias.data = original_resnet.fc.bias.data.clone()
        
    def forward(self, x):
        return self.resnet(x)
    
# 加載預訓練的ResNet50模型
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
#model = MCDropoutResNet()
model = model.to(device)
model.eval()

In [ ]:
# 得到了bzq 正確的函數，拿來做imagenet 正確的預測

def single_data_bzq_mask_preprocessing_imagenet(original_data, start_x, start_y, len_x, len_y, magnification):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data)
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] *= magnification
    #new_data[start_y:start_y + len_y, start_x:start_x + len_x, :] *= magnification
    return new_data

def single_data_bzq_mask_preprocessing_imagenet_random_global(original_data, start_x, start_y, len_x, len_y, random_num_for_bzq_mask_imagenet):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data)
    random_num_for_bzq_mask_imagenet = random_num_for_bzq_mask_imagenet[:, :len_y, :len_x] 
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] = random_num_for_bzq_mask_imagenet
    return new_data

bzq = []
correct_predictions_imagenet = []
incorrect_predictions_imagenet = []
bzq_imagenet = []

#bzq = 0的時候，提取mask之下的softmax
correct_predictions_bzq_zero_softmax_mean = []
correct_predictions_bzq_zero_softmax_std = []
incorrect_predictions_bzq_zero_softmax_mean = []
incorrect_predictions_bzq_zero_softmax_std = []

# 生成隨機影像，形狀為 (limit, 3, 224, 224)
random_images = torch.rand(limit, 3, 224, 224)

# 生成存放 -1 的列表
negative_labels = [-1] * limit

# 建立 test_dataset 為 list of tuples
test_dataset = [(random_images[i], negative_labels[i]) for i in range(limit)]

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [ ]:
# bzq modifying
len_x = len_x_target
len_y = len_y_target
stride_x = stride_x_target
stride_y = stride_y_target
batch_size = 1  # 設定批次大小

original_predictions_imagenet = []
acc_imagenet = []


In [ ]:
import torch
import numpy as np
from collections import Counter
import torch.nn.functional as F
'''
random_num_for_bzq_mask_imagenet = np.random.randint(0, 256, (3, len_y_target, len_x_target)).astype(np.float32) / 255.0
bzq_list = []

#print(random_num_for_bzq_mask_imagenet)

def process_batch(batch_data, len_y, stride_y, len_x, stride_x, device, model, alpha):
    bzq = []

    targets = []
    predictions = []
    for i in range(0, 224 - len_y, stride_y):
        for j in range(0, 224 - len_x, stride_x):
            #print(i, j)
            target = torch.from_numpy(single_data_bzq_mask_preprocessing_imagenet_random_global(batch_data, i, j, len_x, len_y, random_num_for_bzq_mask_imagenet)).float().unsqueeze(0).to(device)
            #print(target)
            output = model(target)
            softmax_output = F.softmax(output, dim=1)  # 轉換成機率
            predictions.append(softmax_output.cpu().numpy())
    
    #print(predictions)
    # 計算 softmax
    predictions = np.array(predictions)
    
    mean_pred = predictions.mean(axis=0)  # 平均預測
    mean_pred = mean_pred.flatten()
    #print(np.max(mean_pred))
    entropy = -np.sum(mean_pred * np.log(np.clip(mean_pred, 1e-10, 1)), axis=0)  # 防止 log(0) 錯誤

    # 計算 bzq
    max_bzq_indices = np.argmax(predictions, axis=1)  # NumPy 陣列

    # **確保是展平的一維列表**
    max_bzq_indices = max_bzq_indices.flatten().tolist()
    counter = Counter(max_bzq_indices) 

    most_common_num, most_common_count = counter.most_common(1)[0]
    temp = predictions[:, most_common_num]

    bzq.append(alpha * np.mean(temp) + (1 - alpha) * (2.0 / np.pi * np.arctan(1.0 / np.std(temp))))

    return bzq, temp, entropy

entropies = []
with torch.no_grad():
    for start in range(0, len(test_dataset), batch_size):
        end = min(start + batch_size, len(test_dataset))
        batch_data_labels = [test_dataset[i] for i in range(start, end)]
        batch_data, batch_labels = zip(*batch_data_labels)
        
        for k in range(len(batch_data)):
            if len(batch_data[k]) > 0:
                bzq, temp, ent = process_batch(
                    batch_data[k], len_y, stride_y, len_x, stride_x, device, model, alpha
                )
                entropies.append(ent)
                

bzq_imagenet = np.array(bzq_list)
'''
# 初始化隨機數陣列
random_num_for_bzq_mask_imagenet = np.random.randint(0, 256, (3, len_y_target, len_x_target)).astype(np.float32) / 255.0
bzq_list = []

def process_batch(batch_data, len_y, stride_y, len_x, stride_x, device, model, alpha):
    """處理 batch 以計算 bzq、temp 及 entropy"""
    predictions = [
        F.softmax(
            model(
                torch.from_numpy(
                    single_data_bzq_mask_preprocessing_imagenet_random_global(
                        batch_data, i, j, len_x, len_y, random_num_for_bzq_mask_imagenet
                    )
                ).float().unsqueeze(0).to(device)
            ),
            dim=1
        ).cpu().numpy()
        for i in range(0, 224 - len_y, stride_y)
        for j in range(0, 224 - len_x, stride_x)
    ]

    predictions = np.array(predictions)

    # 計算 entropy
    mean_pred = predictions.mean(axis=0).flatten()
    entropy = -np.sum(mean_pred * np.log(np.clip(mean_pred, 1e-10, 1)))

    # 計算 bzq
    max_bzq_indices = np.argmax(predictions, axis=1).flatten()
    most_common_num, _ = Counter(max_bzq_indices).most_common(1)[0]
    temp = predictions[:, most_common_num]

    bzq = alpha * np.mean(temp) + (1 - alpha) * (2.0 / np.pi * np.arctan(1.0 / np.std(temp)))

    return bzq, temp, entropy

# 計算 entropy
entropies = []
with torch.no_grad():
    for start in range(0, len(test_dataset), batch_size):
        end = min(start + batch_size, len(test_dataset))
        batch_data_labels = [test_dataset[i] for i in range(start, end)]
        batch_data, _ = zip(*batch_data_labels)

        entropies.extend(
            process_batch(batch_data[k], len_y, stride_y, len_x, stride_x, device, model, alpha)[2]
            for k in range(len(batch_data)) if batch_data[k].numel() > 0  # 確保 Tensor 不為空
        )

bzq_imagenet = np.array(bzq_list)

In [ ]:
print(test_dataset[0][0].size())
# 計算熵的分佈（分桶）
bins = np.linspace(min(entropies), max(entropies), 15) 
hist, bin_edges = np.histogram(entropies, bins=bins)  # 計算直方圖數據
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # 計算每個 bin 的中心點

# 繪製折線圖
plt.figure(figsize=(10, 5))
plt.plot(bin_centers, hist, linestyle='-', marker='o')

# 設定標題與軸標籤
plt.title("Entropy Distribution")
plt.xlabel("Entropy")
plt.ylabel("# of Examples")
plt.legend()
plt.grid(True)
plt.show()